# Chapter 1: Python Data Model
---

## ToC
### [Special Methods](#Special-Methods)
1. [Compare runtime](#Compare-runtime-of-special-and-manual-method)
2. [Emulating Numeric Types](#Emulating-Numeric-Types)
    - 2.1. [String Representation](#String-Representation)
    - 2.2. [Booleav value of a custom type](#boolean-value-of-a-custom-type)
    - 2.3. [Collection API](#Collection-API)
3. [Overview of Special Methods](#overview-of-special-methods)    
---    

## Special Methods

The first thing to know about special methods is that they are meant to be called by
the Python interpreter, and not by you. You don’t write `my_object.__len__()`. You
write `len(my_object)` and, if `my_object` is an instance of a user-defined class, then
Python calls the `__len__` method you implemented.

### Compare runtime of special and manual method

Python variable-sized collections
written in C include a struct2 called PyVarObject, which has an ob_size field holding
the number of items in the collection. So, if my_object is an instance of one of those
built-ins, then len(my_object) retrieves the value of the ob_size field, and this is
much faster than calling a method.

In [2]:
import timeit

# Built-in list
my_list = list(range(1000))

# Custom class with a user-defined method
class MyCollection:
    def __init__(self, items):
        self.items = items

    def size_udf(self):
        return len(self.items)

    def __len__(self):
        return len(self.items)                

my_obj = MyCollection(my_list)

# Time native len() on list
builtin_len_time = timeit.timeit('len(my_list)', globals=globals(), number=1_000_000)

# Time special method via len() on custom object
special_len_time = timeit.timeit('len(my_obj)', globals=globals(), number=1_000_000)

# Time user-defined method call
custom_size_time = timeit.timeit('my_obj.size_udf()', globals=globals(), number=1_000_000)

# print statements
print(f"Time of native len() on list        - len(my_list):       {builtin_len_time:.6f} seconds")
print(f"Time of special method via len()    - len(my_obj):        {special_len_time:.6f} seconds")
print(f"Time of user-defined method call    - my_obj.size_udf():  {custom_size_time:.6f} seconds")


Time of native len() on list        - len(my_list):       0.021010 seconds
Time of special method via len()    - len(my_obj):        0.059026 seconds
Time of user-defined method call    - my_obj.size_udf():  0.032694 seconds


Normally, your code should not have many direct calls to special methods. Unless
you are doing a lot of metaprogramming

### Emulating Numeric Types

![Figure 3](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/3.PNG)


#### Example 1-2. A simple two-dimensional vector class

In [3]:
"""
vector2d.py: a simplistic class demonstrating some special methods

It is simplistic for didactic reasons. It lacks proper error handling,
especially in the ``__add__`` and ``__mul__`` methods.

Addition::
    >>> v1 = Vector(2, 4)
    >>> v2 = Vector(2, 1)
    >>> v1 + v2
    Vector(4, 5)

Absolute value::

    >>> v = Vector(3, 4)
    >>> abs(v)
    5.0

Scalar multiplication::

    >>> v * 3
    Vector(9, 12)
    >>> abs(v * 3)
    15.0
"""

import math

class Vector:

    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        #return 'Vector(%r, %r)' % (self.x, self.y)
        return f"Vector({self.x!r}, {self.y!r})"

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

In [4]:
v1 = Vector(2, 4)
v2 = Vector(2, 1)
v1 + v2

Vector(4, 5)

In [5]:
v = Vector(3, 4)
abs(v)

5.0

In [6]:
v * 3

Vector(9, 12)

In [7]:
abs(v * 3)

15.0

In [8]:
v = Vector(0, 0)
if v:
    print("Vector is non-zero")
else:
    print("Vector is zero")

Vector is zero


In [12]:
v1 * 3

Vector(6, 12)

In [11]:
3 * v1

TypeError: unsupported operand type(s) for *: 'int' and 'Vector'

![Figure 4](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/4.PNG)

### String Representation

Without a custom `__repr__`, Python’s console
would display a Vector instance `<Vector object at 0x10e100070>`.

**Note**: On function think of `x!r` as `f"{x!r}"  ==  f"{repr(x)}"`

- `!r` = use repr(x)

- `!s` = use str(x) (default, so usually you can omit it)

- `!a` = use ascii(x) (for escaped representations)

```python
x = 'hello\nworld'

print(str(x))   # hello
                # world

print(repr(x))  # 'hello\nworld'
```

More about difference between `__str__` and `__repr__` in
Python? [StackOverFlow](https://fpy.li/1-5)

### Boolean Value of a Custom Type

We defined following `__bool__` method:
```python
def __bool__(self):
    return bool(abs(self))
```

![Figure 5](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/5.PNG)

### Collection API

![Figure 6](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/6.PNG)

All the classes in the diagram are ABCs—abstract base classes.

```plaintext
            +-----------------------------------------------+
            |                   Collection                  |  (Python 3.6+)
            |-------------------interfaces------------------|
            | • Iterable: supports `for`, unpacking         |
            | • Sized: supports `len`                       |
            | • Container: supports `in`                    |
            +-----------------------------------------------+
                                    ▲
                   ┌─────────special┼izations───────┐
                   │                │               │
             +-------------+ +-------------+  +-----------+
             |  Sequence   | |  Mapping    |  |  Set      |
             |-------------| |-------------|  |-----------|
             | list, str   | | dict,       |  | set,      |
             | tuple, etc. | | defaultdict |  | frozenset |
             +-------------+ +-------------+  +-----------+
```

Only Sequence is Reversible, because sequences support arbitrary ordering of their
contents, while mappings and sets do not.

![Figure 7](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/7.PNG)

## Overview of Special Methods

![Figure 8](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/8.PNG)

![Figure 9](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/9.PNG)

When Python tries to do a + b, it first checks if a can handle it:

```python
a.__add__(b)
```

If a doesn’t know how to add b (maybe they're different types), then Python tries the reversed version:

```python
b.__radd__(a)
```

![Figure 10](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/10.PNG)

### Why len Is Not a Method

practicality beats purity. `len` is not called as a method because it gets special treatment as part
of the Python Data Model, just like `abs`. But thanks to the special method `__len__`,
you can also make `len` work with your own custom objects.